In [9]:
import requests

Mise en place de l'api

In [10]:
url = "http://api.endlessmedical.com/v1/dx/"

L'api exige qu'avant toute chose on récupère un id session

In [11]:
init_output = requests.get(url +"InitSession")

In [13]:
init_output.json()["SessionID"]

'f7kSms8aan4nEGeO'

In [14]:
session_id = init_output.json()['SessionID']
print(session_id)

f7kSms8aan4nEGeO


Il faut ensuite accepter les terms of use

In [16]:
param_atou = {
    "SessionID":session_id,
    "passphrase":"I have read, understood and I accept and agree to comply with the Terms of Use of EndlessMedicalAPI and Endless Medical services. The Terms of Use are available on endlessmedical.com" 
    }

atou_output = requests.post(url +"AcceptTermsOfUse", params=param_atou)
print(atou_output)
print(atou_output.json())

<Response [200]>
{'status': 'ok'}


Il a 19 ans, il a fait jusqu'à 40 de fièvre la semaine dernière, il a été en contact récent avec quelqu'un de malade.(sa soeur), depuis peu il est devenu végétarien. Il n'a plus d'odorat depuis 3 jours. Il fait de l'asthme mais n'a jamais été hospitalisé pour. 



Patient 2:

Il a 82 ans, il a déja fait un arret cardiaque. Il a eu des épisodes de palpitations, des mots de tête avec une arythmie cardiaque.  Il a aussi mal aux genoux, à l'épaule et aux pieds.  

In [17]:
param = {
    "SessionID":session_id,
    }

patient_one_symptoms = {
    "Age":19,
    "Temp": 104.0,
    "Contacts":3,
    "NewFoods":3,
    "LossOfSmell":3,
    "PMHXAsthma":3
    }


for key,value in patient_one_symptoms.items():
    param_symptom = param.copy()
    param_symptom.update({"name":key, "value":value})
    requests.post(url +"UpdateFeature", params=param_symptom)

In [18]:
patient_on_analysis = requests.get(url +"Analyze", params=param).json()

In [20]:
patient_on_analysis

{'status': 'ok',
 'Diseases': [{'Chronic asthma': '0.28426035261081384'},
  {'Chronic obstructive pulmonary disease (COPD)': '0.04427960901587269'},
  {'Septic shock': '0.04301496341390232'},
  {"Crohn's disease": '0.03611755399695382'},
  {'Ulcerative colitis': '0.031081128501297184'},
  {'Chronic sinusitis': '0.03100216294160063'},
  {'Influenza': '0.02909865152590485'},
  {'Allergic rhinitis to pollen': '0.027735179266990984'},
  {'Coronavirus disease 2019 (Covid-19) mostly upper respiratory tract infection with, or without bronchitis, but without significant pneumonia': '0.025210084033613467'},
  {'Viral pharyngitis (etiology usually rhinovirus, coronavirus, adenovirus, parainfluenza)': '0.02324518850467583'}],
 'VariableImportances': [{'Chronic asthma': [['PMHXAsthma', '1.0'],
    ['FEV1toFVCRatioImprovingBy12BeforeAndAfterBronchodilator',
     '0.8560450022808987'],
    ['DyspneaProgressionSubjective', '0.31910117459501813'],
    ['DyspneaSeveritySubjective', '0.25785600543377946

In [21]:
patient_on_analysis = requests.get(url +"Analyze", params=param).json()

patient_diag = {}
for i in range(3):
    desease_name = list(patient_on_analysis["Diseases"][i].items())[0][0]
    list_symptoms = patient_on_analysis["VariableImportances"][i][desease_name][:2]
    patient_diag[desease_name] = list_symptoms
    
print(patient_diag)

{'Chronic asthma': [['PMHXAsthma', '1.0'], ['FEV1toFVCRatioImprovingBy12BeforeAndAfterBronchodilator', '0.8560450022808987']], 'Chronic obstructive pulmonary disease (COPD)': [['PMHXCOPD', '1.0'], ['pCO2onABG', '0.9066965049828422']], 'Septic shock': [['SBP', '1.0'], ['Arrest', '0.3219868261349471']]}


Pour le deuxième patient, on crée une autre session:

In [58]:
init_output = requests.get(url +"InitSession")
session_id = init_output.json()['SessionID']
param_atou = {
    "SessionID":session_id,
    "passphrase":"I have read, understood and I accept and agree to comply with the Terms of Use of EndlessMedicalAPI and Endless Medical services. The Terms of Use are available on endlessmedical.com" 
    }

atou_output = requests.post(url +"AcceptTermsOfUse", params=param_atou)

param = {
    "SessionID":session_id,
    }

patient_two_symptoms = {
    "Age":82,
    "Arrest": 3,
    "Arrhythmia":4,
    "ArrhythmiaSymptomsHeadaches":4,
    "JointsPain":3,
    }


for key,value in patient_two_symptoms.items():
    param_symptom = param.copy()
    param_symptom.update({"name":key, "value":value})
    requests.post(url +"UpdateFeature", params=param_symptom)

patient_on_analysis = requests.get(url +"Analyze", params=param).json()

patient_diag = {}
for i in range(3):
    desease_name = list(patient_on_analysis["Diseases"][i].items())[0][0]
    list_symptoms = patient_on_analysis["VariableImportances"][i][desease_name][:2]
    patient_diag[desease_name] = list_symptoms
    
print(patient_diag)

{'Septic shock': [['SBP', '1.0'], ['Arrest', '0.3219868261349471']], "Crohn's disease": [['PMHXChrons', '1.0'], ['CrohnsOnColonoscopy', '0.47406799428819785']], 'Arterial embolism (embolization) caused lower extremity ischemia': [['ArterialLowerExtremitiesDopplers', '1.0'], ['DistalPulsesLE', '0.6980399762010554']]}


Le meme résultat en factorisant:

In [23]:
def diagnostic_function(dict_symptoms):
    init_output = requests.get(url +"InitSession")
    session_id = init_output.json()['SessionID']
    param_atou = {
        "SessionID":session_id,
        "passphrase":"I have read, understood and I accept and agree to comply with the Terms of Use of EndlessMedicalAPI and Endless Medical services. The Terms of Use are available on endlessmedical.com" 
        }

    requests.post(url +"AcceptTermsOfUse", params=param_atou)

    param = {
        "SessionID":session_id,
        }


    for key,value in dict_symptoms.items():
        param_symptom = param.copy()
        param_symptom.update({"name":key, "value":value})
        requests.post(url +"UpdateFeature", params=param_symptom)

    patient_on_analysis = requests.get(url +"Analyze", params=param).json()

    patient_diag = {}
    for i in range(3):
        desease_name = list(patient_on_analysis["Diseases"][i].items())[0][0]
        list_symptoms = patient_on_analysis["VariableImportances"][i][desease_name][:2]
        patient_diag[desease_name] = list_symptoms

    return patient_diag


     

In [24]:
patient_one_symptoms = {
    "Age":19,
    "Temp": 104.0,
    "Contacts":3,
    "NewFoods":3,
    "LossOfSmell":3,
    "PMHXAsthma":3
    }

patient_two_symptoms = {
    "Age":82,
    "Arrest": 3,
    "Arrhythmia":4,
    "ArrhythmiaSymptomsHeadaches":4,
    "JointsPain":3,
    }

print(diagnostic_function(patient_one_symptoms))
print(diagnostic_function(patient_two_symptoms))

{'Chronic asthma': [['PMHXAsthma', '1.0'], ['FEV1toFVCRatioImprovingBy12BeforeAndAfterBronchodilator', '0.8560450022808987']], 'Chronic obstructive pulmonary disease (COPD)': [['PMHXCOPD', '1.0'], ['pCO2onABG', '0.9066965049828422']], 'Septic shock': [['SBP', '1.0'], ['Arrest', '0.3219868261349471']]}
{'Septic shock': [['SBP', '1.0'], ['Arrest', '0.3219868261349471']], "Crohn's disease": [['PMHXChrons', '1.0'], ['CrohnsOnColonoscopy', '0.47406799428819785']], 'Arterial embolism (embolization) caused lower extremity ischemia': [['ArterialLowerExtremitiesDopplers', '1.0'], ['DistalPulsesLE', '0.6980399762010554']]}
